In [ ]:
%pip install sacremoses==0.0.53
%pip install datasets
%pip install transformers
%pip install torch torchvision torchaudio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 880.6/880.6 kB 12.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895239 sha256=a0d1ec1cd0bce656db466b114a3e32bf222d81b5be393188be367bc716fbfdde
  Stored in directory: /root/.cache/pip/wheels/00/24/97/a2ea5324f36bc626e1ea0267f33db6aa80d157ee977e9e42fb
Successfully built sacremoses
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.9 MB/s eta 0:00:00


In [ ]:
!pip install transformers torch accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 5.3 MB/s eta 0:00:00


In [ ]:
!pip install accelerate

In [ ]:
from keras.preprocessing.text import Tokenizer

import pandas as pd
import numpy as np
import string
import nltk
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.lm.preprocessing import padded_everygram_pipeline
from nltk.lm import MLE
from nltk.lm import Vocabulary
from transformers import T5ForConditionalGeneration, T5Tokenizer

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')
nltk.download('words')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Package words is already up-to-date!


True

In [ ]:
texts = []
news_df = pd.read_csv('https://raw.githubusercontent.com/sunnysai12345/News_Summary/master/news_summary_more.csv',nrows =5000)
texts.extend(list(news_df.text.values))
texts = [h for h in texts if h != "Unknown"]
len(texts)

5000

In [ ]:
texts

["Saurav Kant, an alumnus of upGrad and IIIT-B's PG Program in Machine learning and Artificial Intelligence, was a Sr Systems Engineer at Infosys with almost 5 years of work experience. The program and upGrad's 360-degree career support helped him transition to a Data Scientist at Tech Mahindra with 90% salary hike. upGrad's Online Power Learning has powered 3 lakh+ careers.",
 "Kunal Shah's credit card bill payment platform, CRED, gave users a chance to win free food from Swiggy for one year. Pranav Kaushik, a Delhi techie, bagged this reward after spending 2000 CRED coins. Users get one CRED coin per rupee of bill paid, which can be used to avail rewards from brands like Ixigo, BookMyShow, UberEats, Cult.Fit and more.",
 "New Zealand defeated India by 8 wickets in the fourth ODI at Hamilton on Thursday to win their first match of the five-match ODI series. India lost an international match under Rohit Sharma's captaincy after 12 consecutive victories dating back to March 2018. The ma

In [ ]:
texts = [str(s) for s in texts]
texts

["Saurav Kant, an alumnus of upGrad and IIIT-B's PG Program in Machine learning and Artificial Intelligence, was a Sr Systems Engineer at Infosys with almost 5 years of work experience. The program and upGrad's 360-degree career support helped him transition to a Data Scientist at Tech Mahindra with 90% salary hike. upGrad's Online Power Learning has powered 3 lakh+ careers.",
 "Kunal Shah's credit card bill payment platform, CRED, gave users a chance to win free food from Swiggy for one year. Pranav Kaushik, a Delhi techie, bagged this reward after spending 2000 CRED coins. Users get one CRED coin per rupee of bill paid, which can be used to avail rewards from brands like Ixigo, BookMyShow, UberEats, Cult.Fit and more.",
 "New Zealand defeated India by 8 wickets in the fourth ODI at Hamilton on Thursday to win their first match of the five-match ODI series. India lost an international match under Rohit Sharma's captaincy after 12 consecutive victories dating back to March 2018. The ma

In [ ]:
def preprocessing(text):
    # Lowercase
    text = text.lower()
    # Punctuations
    text_p = "".join([char for char in text if char not in string.punctuation])
    # Tokenise
    words = word_tokenize(text_p)
    stop_words = stopwords.words('english')
    # Remove stop words
    filtered_words = [word for word in words if word not in stop_words]
    # Stemming
    porter = PorterStemmer()
    stemmed = [porter.stem(word) for word in filtered_words]
    return stemmed


corpus = [preprocessing(x) for x in texts]
texts[0],corpus[0]

("Saurav Kant, an alumnus of upGrad and IIIT-B's PG Program in Machine learning and Artificial Intelligence, was a Sr Systems Engineer at Infosys with almost 5 years of work experience. The program and upGrad's 360-degree career support helped him transition to a Data Scientist at Tech Mahindra with 90% salary hike. upGrad's Online Power Learning has powered 3 lakh+ careers.",
 ['saurav',
  'kant',
  'alumnu',
  'upgrad',
  'iiitb',
  'pg',
  'program',
  'machin',
  'learn',
  'artifici',
  'intellig',
  'sr',
  'system',
  'engin',
  'infosi',
  'almost',
  '5',
  'year',
  'work',
  'experi',
  'program',
  'upgrad',
  '360degre',
  'career',
  'support',
  'help',
  'transit',
  'data',
  'scientist',
  'tech',
  'mahindra',
  '90',
  'salari',
  'hike',
  'upgrad',
  'onlin',
  'power',
  'learn',
  'power',
  '3',
  'lakh',
  'career'])

In [ ]:
tokenizer = Tokenizer()

def get_sequence_of_tokens(corpus):
    ## tokenization
    tokenizer.fit_on_texts(corpus)
    total_words = len(tokenizer.word_index) + 1

    ## convert data to sequence of tokens
    text_sequences = []
    for line in corpus:
        token_list = tokenizer.texts_to_sequences([line])[0]
        text_sequences.append(token_list)
    return text_sequences, total_words

text_sequences, total_words = get_sequence_of_tokens(corpus)
text_sequences[0],total_words

([9967,
  9968,
  4912,
  1998,
  7176,
  4913,
  1461,
  1520,
  754,
  1521,
  933,
  5787,
  304,
  514,
  2091,
  1050,
  689,
  4,
  42,
  834,
  1461,
  1998,
  4323,
  862,
  217,
  117,
  2092,
  232,
  550,
  3518,
  1829,
  1912,
  1579,
  2202,
  1998,
  260,
  291,
  754,
  291,
  536,
  120,
  862],
 18996)

In [ ]:

# N-gram
def Ngram(corpus,n):
    out = []
    for seq in corpus:
        for i in range(len(seq)-n+1):
            out.append(seq[i:i+n])
    return out

trigram = Ngram(text_sequences, 3)
trigram[:20]

[[9967, 9968, 4912],
 [9968, 4912, 1998],
 [4912, 1998, 7176],
 [1998, 7176, 4913],
 [7176, 4913, 1461],
 [4913, 1461, 1520],
 [1461, 1520, 754],
 [1520, 754, 1521],
 [754, 1521, 933],
 [1521, 933, 5787],
 [933, 5787, 304],
 [5787, 304, 514],
 [304, 514, 2091],
 [514, 2091, 1050],
 [2091, 1050, 689],
 [1050, 689, 4],
 [689, 4, 42],
 [4, 42, 834],
 [42, 834, 1461],
 [834, 1461, 1998]]

In [ ]:

# POS Tagging
pos = []
for seq in corpus[:10]:
    pos.append(nltk.pos_tag(seq))

pos[0]

[('saurav', 'NN'),
 ('kant', 'NN'),
 ('alumnu', 'NN'),
 ('upgrad', 'JJ'),
 ('iiitb', 'JJ'),
 ('pg', 'NN'),
 ('program', 'NN'),
 ('machin', 'NN'),
 ('learn', 'VBP'),
 ('artifici', 'JJ'),
 ('intellig', 'NN'),
 ('sr', 'NN'),
 ('system', 'NN'),
 ('engin', 'NN'),
 ('infosi', 'VBZ'),
 ('almost', 'RB'),
 ('5', 'CD'),
 ('year', 'NN'),
 ('work', 'NN'),
 ('experi', 'NN'),
 ('program', 'NN'),
 ('upgrad', 'JJ'),
 ('360degre', 'CD'),
 ('career', 'NN'),
 ('support', 'NN'),
 ('help', 'NN'),
 ('transit', 'VB'),
 ('data', 'NNS'),
 ('scientist', 'NN'),
 ('tech', 'NN'),
 ('mahindra', 'VBD'),
 ('90', 'CD'),
 ('salari', 'NNS'),
 ('hike', 'JJ'),
 ('upgrad', 'JJ'),
 ('onlin', 'NN'),
 ('power', 'NN'),
 ('learn', 'VBP'),
 ('power', 'NN'),
 ('3', 'CD'),
 ('lakh', 'NNS'),
 ('career', 'NN')]

In [ ]:
tokenized_text = corpus
n = 2
train_data = [nltk.bigrams(t) for t in tokenized_text]
words = [word for sent in tokenized_text for word in sent]
words.extend(["<s>", "</s>"])
padded_vocab = Vocabulary(words)
model = MLE(n)
model.fit(train_data, padded_vocab)

test_sentences = ['pm modi', 'saurav kant']

tokenized_text = [preprocessing(i) for i in test_sentences]

test_data = [nltk.bigrams(t) for t in tokenized_text]

for test in test_data:
    print ("MLE Estimates:", [((ngram[-1], ngram[:-1]), model.score(ngram[-1], ngram[:-1])) for ngram in test])

test_data = [nltk.bigrams(t) for t in tokenized_text]
for i, test in enumerate(test_data):
  print("PP({0}):{1}".format(test_sentences[i], model.perplexity(test)))

MLE Estimates: [(('modi', ('pm',)), 0.1980440097799511)]
MLE Estimates: [(('kant', ('saurav',)), 1.0)]
PP(pm modi):5.049382716049383
PP(saurav kant):1.0


In [ ]:
for sent in texts[:10]:
    for chunk in nltk.ne_chunk(nltk.pos_tag(nltk.word_tokenize(sent))):
        if hasattr(chunk, 'label'):
            print(chunk.label(), '-', ' '.join(c[0] for c in chunk))

GPE - Saurav Kant
ORGANIZATION - upGrad
ORGANIZATION - PG Program
GPE - Machine
ORGANIZATION - Artificial Intelligence
ORGANIZATION - Sr Systems Engineer
ORGANIZATION - Infosys
ORGANIZATION - upGrad
FACILITY - Tech Mahindra
ORGANIZATION - upGrad
PERSON - Online Power Learning
PERSON - Kunal
ORGANIZATION - Shah
ORGANIZATION - CRED
GPE - Swiggy
PERSON - Pranav Kaushik
ORGANIZATION - CRED
ORGANIZATION - CRED
PERSON - Ixigo
ORGANIZATION - BookMyShow
ORGANIZATION - UberEats
GPE - New
PERSON - Zealand
GPE - India
ORGANIZATION - Hamilton
ORGANIZATION - ODI
GPE - India
PERSON - Rohit Sharma
GPE - India
ORGANIZATION - ODI
PERSON - Aegon Life
GPE - Insurance
GPE - Critical
ORGANIZATION - Disability
ORGANIZATION - Accidental Death Benefit
PERSON - Rajkumar Hirani
PERSON - Sonam Kapoor
PERSON - Hirani
ORGANIZATION - MeToo
ORGANIZATION - MeToo
PERSON - Hirani
GPE - Pakistani
PERSON - Rahat Fateh Ali Khan
ORGANIZATION - Enforcement Directorate
GPE - India
GPE - Rahat
GPE - India
ORGANIZATION - ODI
G

#T5 small


In [ ]:
from transformers import pipeline

summarizer = pipeline(
    task="summarization",
    model="t5-small",
    min_length=20,
    max_length=40,
    truncation=True,
    model_kwargs={"cache_dir": '/Documents/Huggin_Face/'},
)

for input_text in texts[:20]:
    summary = summarizer(input_text, max_length=150, min_length=30, do_sample=False)[0]['summary_text']

    bullet_points = summary.split(". ")

    print(f"\nInput Text:\n{input_text}\n")
    print("Generated Summary:")
    for point in bullet_points:
        print(f"- {point}")




config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

...


Input Text:
Filmmaker Karan Johar and actress Tabu turned showstoppers for Gaurav Gupta on the opening night of LakmÃ© Fashion Week Summer/ Resort 2019. While Johar wore a red sequinned jacket with black pants, Tabu walked the ramp in a grey embellished gown. The fashion show, which began on January 29, will continue till February 3.

Generated Summary:
- filmmaker and actress turned showstoppers for Gaurav Gupta on the opening night of lakm fashion week 
- the fashion show, which began on January 29, will continue till February 3 .


Your max_length is set to 150, but your input_length is only 106. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=53)



Input Text:
In a jibe at Congress President Rahul Gandhi, PM Narendra Modi on Wednesday said those on "bail will have to go to jail." PM Modi added, "He is out on bail and his associates too are facing charges...I know they will be convicted one day." The PM claimed he'd waged a war on corruption because he's from a common household. 

Generated Summary:
- "he is out on bail and his associates too are facing charges," he adds 
- the PM claimed he waged a war on corruption because he's from a common household .


Your max_length is set to 150, but your input_length is only 94. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=47)



Input Text:
Days after he threatened to step down from his post if Congress MLAs continue "crossing the line," Karnataka Chief Minister HD Kumaraswamy accused them of taking potshots and asked, "How many more days can I tolerate such stuff?" Kumaraswamy, who made the statements after a Congress MLA demanded that Siddaramaiah be made CM again, said, "Power is ephemeral."

Generated Summary:
- HD Kumaraswamy threatened to step down from his post if Congress MLAs continue "crossing the line" he said, "Power is ephemeral"

Input Text:
Union Minister Dharmendra Pradhan on Wednesday claimed the illegal mining mafia in Odisha operates under the control of CM Naveen Patnaik and state Congress chief Niranjan Patnaik. He added, "The time has come for the people of Odisha to put a full stop to their activities...The time has come for us to ask for an explanation from this corrupt government."

Generated Summary:
- union minister says the mining mafia in Odisha operates under the control of CM Na

#Bart


In [ ]:
from transformers import BartForConditionalGeneration, BartTokenizer

tokenizer_bert = BartTokenizer.from_pretrained('facebook/bart-large-cnn')
model_bert = BartForConditionalGeneration.from_pretrained('facebook/bart-large-cnn')

def summarize_with_bert(text):
    inputs = tokenizer_bert.encode("summarize: " + text, return_tensors="pt", max_length=512, truncation=True)
    outputs = model_bert.generate(inputs, max_length=20, length_penalty=2.0, num_beams=4, early_stopping=True)
    summarized_text = tokenizer_bert.decode(outputs[0], skip_special_tokens=True)
    return summarized_text

bert_summaries = []

for input_text in texts[:10]:
    summary_bert = summarize_with_bert(input_text)
    bert_summaries.append(summary_bert)

for i, bert_summary in enumerate(bert_summaries):
    print(f"\nInput Text {i + 1}:\n{texts[i]}\n")
    print("BERT Generated Summary:")
    print(f"- {bert_summary}\n")



/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1298: UserWarning: Unfeasible length constraints: `min_length` (56) is larger than the maximum possible length (20). Generation will stop at the defined maximum length. You should decrease the minimum length and/or increase the maximum length.
  warnings.warn(



Input Text 1:
Saurav Kant, an alumnus of upGrad and IIIT-B's PG Program in Machine learning and Artificial Intelligence, was a Sr Systems Engineer at Infosys with almost 5 years of work experience. The program and upGrad's 360-degree career support helped him transition to a Data Scientist at Tech Mahindra with 90% salary hike. upGrad's Online Power Learning has powered 3 lakh+ careers.

BERT Generated Summary:
- Saurav Kant, an alumnus of upGrad and IIIT-B


Input Text 2:
Kunal Shah's credit card bill payment platform, CRED, gave users a chance to win free food from Swiggy for one year. Pranav Kaushik, a Delhi techie, bagged this reward after spending 2000 CRED coins. Users get one CRED coin per rupee of bill paid, which can be used to avail rewards from brands like Ixigo, BookMyShow, UberEats, Cult.Fit and more.

BERT Generated Summary:
- Pranav Kaushik, a Delhi techie, bagged this reward after


Input Text 3:
New Zealand defeated India by 8 wickets in the fourth ODI at Hamilton on 

In [ ]:
# Python program to generate word vectors using Word2Vec

# importing all necessary modules
from nltk.tokenize import sent_tokenize, word_tokenize
import warnings

warnings.filterwarnings(action = 'ignore')

import gensim
from gensim.models import Word2Vec

# Reads ‘modi.txt’ file
# sample = open("C:\\Users\\Admin\\Desktop\\modi.txt", "utf8")
# s = sample.read()

# # Replaces escape character with space
# f = s.replace("\n", " ")

data = corpus

# iterate through each sentence in the file
# for i in sent_tokenize(f):
# 	temp = []

# 	# tokenize the sentence into words
# 	for j in word_tokenize(i):
# 		temp.append(j.lower())

# 	data.append(temp)

# Create CBOW model
model1 = gensim.models.Word2Vec(data, min_count = 1,
							vector_size = 100, window = 5)

# Print results
print("Cosine similarity between 'modi' " +
			"and 'pm' - CBOW : ",
	model1.wv.similarity('modi', 'pm'))

print("Cosine similarity between 'modi' " +
				"and 'kant' - CBOW : ",
	model1.wv.similarity('modi', 'kant'))

# Create Skip Gram model
model2 = gensim.models.Word2Vec(data, min_count = 1, vector_size = 100,
											window = 5, sg = 1)

# Print results
print("Cosine similarity between 'modi' " +
		"and 'pm' - Skip Gram : ",
	model2.wv.similarity('modi', 'pm'))

print("Cosine similarity between 'modi' " +
			"and 'kant' - Skip Gram : ",
	model2.wv.similarity('modi', 'kant'))


Cosine similarity between 'modi' and 'pm' - CBOW :  0.99583286
Cosine similarity between 'modi' and 'kant' - CBOW :  0.6690802
Cosine similarity between 'modi' and 'pm' - Skip Gram :  0.9693166
Cosine similarity between 'modi' and 'kant' - Skip Gram :  0.5661483
